In [ ]:
# This code is how you download the NOAA Satellite Images. (You will need these files so that you can apply RGB to them.)
# Make sure you have the AMS 2023 Environment installed
# I use the dust storm of Decmeber 15 2021 as an example
# Three ### will represent something that needs changed in the code to get a different image

In [ ]:
# Import these libraries so that the code runs smoothly

import s3fs

import requests

import datetime

import numpy as np

from pathlib import Path

In [ ]:
# Update your file directory

import os
os.chdir('/Users/cires/Documents/Research/RECCS_2023_Ethan/AMS_Short_Course_2023_Python_Files/Other_Dust_Storms') # This is an example you will need to change your directory so that it is relevant to you

In [ ]:
# Ensure that your file directory path is correct for the satellite images

directory_path = Path.cwd()
print(directory_path)

In [ ]:
# Activate this package

fs = s3fs.S3FileSystem(anon=True)

In [ ]:
# The GOES Satellite that views all of America is GOES 16. (We will zoom in on these files later)

bucket = 'noaa-goes16'

products_path = bucket

products = fs.ls(products_path)

for product in products:
    print(product.split('/')[-1])

In [ ]:
# Change the below times to the Day, Month, and Year you would like to view

year = 2021 ###
month = 12  ###
day = 15    ###

julian_day = datetime.datetime(year, month, day).strftime('%j')
print(julian_day)

In [ ]:
# The hour is in UTC be sure to convert this to the reigion you are in
# This code will download the images you need. You will need to convert hour to your time region
# The product I have chosen views all of the United States. (You will be able to zoom into your area of intrest later)

bucket = 'noaa-goes16'
product = 'ABI-L2-MCMIPC'
year = 2021                                      ###
julian = julian_day
hour = 20                                        ###

data_path = bucket + '/' + product + '/'  + str(year) + '/' + str(julian).zfill(3) + '/' + str(hour).zfill(2)

files = fs.ls(data_path)

print('Total number of files:', len(files), '\n')

print('Print out the names of the first 10 and last 10 files')
for file in files[:10]:
    print(file.split('/')[-1])
for file in files[-10:]:
    print(file.split('/')[-1])

In [ ]:
# Select how long you want to view the images for
# Simplified Example: observation_times 20:01 to 20:57 every 5 minutes (See below)

observation_times = np.arange(2001,2057,5).astype(str) ###
product_name = 'MCMIPC'
for observation_time in observation_times:
    print(observation_time)
    matches = [file for file in files if (file.split('/')[-1].split('_')[3][8:12] == observation_time and file.split('/')[-1].split('-')[2] == product_name)]

    for match in matches:
        print(match.split('/')[-1])
        print('Approximate file size (MB):', round((fs.size(match)/1.0E6), 2))
    ## fs.get is the command that downloads the file

    for match in matches:
        print(match)
        print(str(directory_path / match.split('/')[-1]))
        fs.get(match, str(directory_path / match.split('/')[-1]))

In [ ]:
# It will take awhile to generate all of the files you need
# You should have all of the files you need
# Now we can start working on adding RGB and enhancing your area of intrest

In [ ]:
# Import these libraries:

import xarray as xr

import numpy as np

from matplotlib import pyplot as plt

from cartopy import crs as ccrs
import cartopy.feature as cfeature

import datetime

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Make sure you have the same path as above

directory_path = Path.cwd()

In [ ]:
# This part is kind if annoying. Enter the file you want to enhance (At the moment you have to do this step 1 at a time)
# If someone knows how to do and eneter all of the files at once please add on
# If I have time I will try to make it more automatic

file_name = '' ###

file_id = directory_path / file_name

ds = xr.open_dataset(file_id, engine='netcdf4')
ds

In [ ]:
# For the next two steps you need to read in metdata 1 and 13

ds.CMI_C01.attrs

In [ ]:
ds.CMI_C13.attrs

In [ ]:
# Lets start by calibrating the RGB for your image
# top left | top right | bottom left | bottom right

red   = np.array( [ [ 1.0, 1.0 ], [ 0.0, 0.0] ] )
green = np.array( [ [ 0.0, 1.0 ], [ 0.0, 1.0] ] )
blue  = np.array( [ [ 0.0, 0.0 ], [ 1.0, 1.0] ] )

rgb = np.dstack([red, green, blue])

print(rgb.shape)

In [ ]:
# Here is an example RGB

plt.figure()
plt.imshow(rgb)
plt.show()

In [ ]:
# Ensure that are of your values are between 1 and 0

print(ds['CMI_C01'].values.min(), ds['CMI_C01'].values.max())
print(ds['CMI_C02'].values.min(), ds['CMI_C02'].values.max())
print(ds['CMI_C03'].values.min(), ds['CMI_C03'].values.max())

In [ ]:
# Check the shape of your file

ds['CMI_C01'].shape, ds['CMI_C02'].shape, ds['CMI_C03'].shape

In [ ]:
# If you need to change the shape of your file do it here

ch1 = ds['CMI_C01']
ch1_resize = ch1[::2,::2]
ch1.shape, ch1_resize.shape

In [ ]:
# Here is your current shape

ch1_coarsen = ch1.coarsen(x=2, y=2).mean()
ch1_coarsen.shape

In [ ]:
# Ensure these data arrays are in your dataset

ch1 = ds.CMI_C01
ch2 = ds.CMI_C02
ch3 = ds.CMI_C03

In [ ]:
# The next 3 steps make green for your RGB image

green = 0.45*ch2 + 0.1*ch3 + 0.45*ch1

In [ ]:
green.values.min(), green.values.max()

In [ ]:
tc_RGB = np.dstack([ch2, green, ch1])
print(tc_RGB.shape)

In [ ]:
# Here is what your image looks like with minimal RGB

plt.figure()
plt.imshow(tc_RGB)
plt.show()

In [ ]:
# This code helps with gamma luminance

gamma = 2.5
tc_RGB_gamma = np.power(tc_RGB, 1/gamma)

plt.figure()
plt.imshow(tc_RGB_gamma)
plt.show()

In [ ]:
# In order to see dust you will need to extract these channels

ch11 = ds.CMI_C11
ch13 = ds.CMI_C13
ch14 = ds.CMI_C14
ch15 = ds.CMI_C15

In [ ]:
# To make red appear pink you must subtract these channels

img = ch15-ch13
print(np.min(img.values))
print(np.max(img.values))

In [ ]:
# Now normalize the red layers

lower_val = -6.7
upper_val = 2.6

# np.clip -- use it to set all data above 2.6 to 1.0 and all data below -6.7 to 0:
img_clip = np.clip(img, lower_val, upper_val)
print(np.min(img_clip.values))
print(np.max(img_clip.values))

# Normalizes the clipped data
normalized_red = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Ensure your RGB values are in range

normalized_red.values.min(), normalized_red.values.max()

In [ ]:
# Normalize the green layers

img = ch14-ch11

lower_val = -0.5
upper_val = 20.0

img_clip = np.clip(img, lower_val, upper_val)
normalized_green = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Gamma corrections for the green layer

gamma = 2.5
normalized_green_gamma = np.power(normalized_green, 1/gamma)

In [ ]:
# Normalize the blue layer

img = ch13

lower_val = 261.2
upper_val = 288.7

img_clip = np.clip(img, lower_val, upper_val)
normalized_blue = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Stack the Red, Blue and Green layers

dust_RGB = np.dstack([normalized_red, normalized_green_gamma, normalized_blue])

In [ ]:
# Lets see what the image looks like

plt.figure()
plt.imshow(dust_RGB)
plt.show()

In [ ]:
# Define channels
ch8 = ds.CMI_C08
ch10 = ds.CMI_C10
ch12 = ds.CMI_C12
ch13 = ds.CMI_C13

In [ ]:
# Red
img = ds.CMI_C08 - ds.CMI_C10

lower_val = -26.2
upper_val = 0.6

img_clip = np.clip(img, lower_val, upper_val)
normalized_red = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Green
img = ds.CMI_C12 - ds.CMI_C13

lower_val = -43.2
upper_val = 6.7

img_clip = np.clip(img, lower_val, upper_val)
normalized_green = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Blue
img = ds.CMI_C08

lower_val = 208.5
upper_val = 244.0

img_clip = np.clip(img, lower_val, upper_val)
normalized_blue = (img_clip-lower_val)/(upper_val-lower_val)
normalized_blue_inverted = 1-normalized_blue

In [ ]:
airmass_RGB = np.dstack([normalized_red, normalized_green, normalized_blue_inverted])

In [ ]:
plt.figure()
plt.imshow(airmass_RGB)
plt.show()

In [ ]:
# Lets touch up that image

proj_var = ds.goes_imager_projection

sat_height = proj_var.perspective_point_height
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis

globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)

In [ ]:
# Define the native geostationary map projection

central_lon = proj_var.longitude_of_projection_origin
print(central_lon)
crs = ccrs.Geostationary(central_longitude=central_lon,satellite_height=sat_height, globe=globe)

In [ ]:
# Define the extent of the image

X = ds['x']*sat_height
Y = ds['y']*sat_height
imgExtent = (X.min(), X.max(), Y.min(), Y.max())

In [ ]:
# Lets add some features to make the map easier to naviagte

proj_to = crs

fig = plt.figure()
ax = plt.subplot(projection=proj_to)

ax.coastlines('10m', linewidth=2)
ax.imshow(dust_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.show()

In [ ]:
# Lets add a title to your map

ds.attrs

In [ ]:
#Lets add the time and date that this image was taken

platform = ds.platform_ID + ' ' + ds.title[0:3]

dtinfo_s = ds.time_coverage_start[0:16].replace('T',' ')
dtinfo_e = ds.time_coverage_end[0:16].replace('T',' ')

dt_scan = datetime.datetime.strptime(dtinfo_s, '%Y-%m-%d %H:%M')
date_s = dt_scan.strftime('%d %b %Y')
print(date_s)
time_s = dt_scan.strftime('%H:%M')
print(time_s)

composite = (f'Dust RGB Composite {date_s}')
formula = r'$Red = BT_{12.3\mu m}-BT_{10.3\mu m} \ \ \ Green = BT_{11.2\mu m}-BT_{8.4\mu m} \ \ \ Blue = BT_{10.3\mu m}$'

plot_title = platform + ' ' + composite + ' ' + time_s

In [ ]:
#Here is what your title will look like
plot_title

In [ ]:
# This last step zooms into your area of intrest and downloads your enhanced image
# It is kind of a guessing game until you have your area of intrest

proj_to = crs

fig = plt.figure(figsize=(5,5))
ax = plt.subplot(projection=proj_to)
ax.set_xlim((-2600000.5, -1481770.0))  # Change this to zoom in vertically   ###
ax.set_ylim((3584175.875, 4288198.0))  # Change this to zoom in horizontally ###

ax.coastlines('10m', linewidth=2)
ax.imshow(dust_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.title(plot_title, size=8)

plt.show()

saved_file_name=file_name+'.png'
fig.savefig(saved_file_name, facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
# How to make a gif image using the png you have just created

In [ ]:
pip install pillow

In [ ]:
# Insert your files wherever there is .png
# I may have given you to many frames (You can # out the extras)

from PIL import Image

frames = []

frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open(""))
frames.append(Image.open("OR_ABI-L2-MCMIPC-M6_G16_s20213492051172_e20213492053545_c20213492054050.nc.png"))
frames.append(Image.open("OR_ABI-L2-MCMIPC-M6_G16_s20213492056172_e20213492058545_c20213492059037.nc.png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))
#frames.append(Image.open(".png"))

frames[0].save("December 15, 2021 Dust_Storm.gif", save_all=True, append_images=frames[1:], optimize=False, duration=200, loop=0)  ### Change this to the proper dust storm name

In [ ]:
# Congrats your gif should have been made